In [37]:
# CURRENTLY WORKING IN THIS ONE
import os
import torch 
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
from torchvision import datasets
from torch.utils.data import DataLoader
from torchvision.utils import save_image
from PIL import Image
import numpy as np
from random import randint
from sklearn import preprocessing as p
import gc
import cv2
import math


In [38]:
torch.cuda.empty_cache()
gc.collect()

0

In [39]:
min_max_scaler = p.MinMaxScaler()
INPUT_DATA_PATH = 'test_input_new.npy'
OUTPUT_DATA_PATH = 'test_output_new.npy'
x_input = np.load(INPUT_DATA_PATH)
y_input = np.load(OUTPUT_DATA_PATH)

x_min = x_input.min(axis=(1, 2), keepdims=True)
x_max = x_input.max(axis=(1, 2), keepdims=True)
x_input = (x_input - x_min)/(x_max-x_min)

y_min = y_input.min(axis=(1, 2), keepdims=True)
y_max = y_input.max(axis=(1, 2), keepdims=True)
y_input = (y_input - y_min)/(y_max-y_min)

num_points = np.load('test_nums.npy')
# print(x_input)

In [40]:
# x_input[x_input < 1] = 0


# print(x_input[0][0])

In [41]:
def threshold(tensor):
    for img in range(len(tensor)):
            for channel in range(1):
                    for h in range(100):
                            for w in range(100):
                                    if (tensor[img][channel][h][w] < 1):
                                            tensor[img][channel][h][w] = 0

In [42]:
SPLIT_IDX = 7500
x_train = torch.tensor(x_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
x_test = torch.tensor(x_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

y_train = torch.tensor(y_input[:SPLIT_IDX], dtype=torch.float).permute(0,3,1,2)
y_test = torch.tensor(y_input[SPLIT_IDX:], dtype=torch.float).permute(0,3,1,2)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
x_train = x_train.to(device)
x_test = x_test.to(device)
y_train = y_train.to(device)
y_test = y_test.to(device)

test_num_points = num_points[SPLIT_IDX:]

# print (test_num_points.shape)

# threshold(x_train)                       
# threshold(x_test)
# threshold(y_train)                       
# threshold(y_test)


train_dataset = TensorDataset(x_train,y_train) # create your datset
# train_dataloader = DataLoader(train_dataset, shuffle=True) # create your dataloader

train_dataloader = DataLoader(train_dataset, batch_size=100, shuffle=True) # create your dataloader

print (len(train_dataloader))

test_dataset = TensorDataset(x_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=100, shuffle=False)

#/home/bwilab/asha_ritu/line_model/conv_network.ipynb


75


In [43]:
# create a CNN
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()

        self.cnn_stack = nn.Sequential( 
            # first layer: learning one person. 
            # kernel size = diameter of a dot (5 pixels) + padding (2 pixels) = (7,7)
            # out_channels: 1 for now
            # try changing the out_channels
            # larger dataset and ReLU between each layer
            # larger dataset, more layers, larger kernels

            # find center pixel of each circle, count number of dots

            # blob detect: looking for continuous regions of non-white pixels
            # confidence regions. 
            # masking the image like in hw3
            # 
            # nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(2,2), stride=1, bias=True, padding=(0,0)), # input layer
            # nn.ReLU(),

            # nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(11,11), stride=1, bias=True, padding=(5,5)), # input layer
            # nn.ReLU(),

            # nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(19,19), stride=1, bias=True, padding=(9,9)), # input layer nn.Conv2d(in_channels=10, out_channels=1, kernel_size=(6,6), padding=(3,3)), # 100 x 100
            nn.ReLU()
            # nn.Conv2d(in_channels=10, out_channels=1, kernel_size=(6,6), padding=(3,3)), # 100 x 100
            # nn.ReLU()
            
            nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(6,6), stride=1, bias=True, padding=(2,2)), # input layer
            nn.ReLU(),

            # nn.MaxPool2d(kernel_size=(2,2), stride=2),

            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(11,11), stride=1, bias=True, padding=(5,5)), # input layer
            nn.ReLU(),

            # nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(19,19), stride=1, bias=True, padding=(9,9)), # input layer
            # nn.ReLU(),
            nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(11,11), stride=1, bias=True, padding=(5,5)), # input layer
            nn.ReLU(),

            nn.Conv2d(in_channels=10, out_channels=1, kernel_size=(6,6), padding=(3,3)), # 100 x 100
            nn.ReLU()

        )

    def forward(self, x):
        logits = self.cnn_stack(x)
        return logits

In [44]:
model = NeuralNetwork()
if torch.cuda.is_available():
    model.cuda()
    
learning_rate = 3e-4
weight_decay=1e-5
optimizer = torch.optim.Adam(params=model.parameters(), weight_decay=weight_decay, lr=learning_rate)
batch_size = 16
epochs = 30
# Initialize the loss function
loss_fn = nn.MSELoss()
# loss_fn = nn.CrossEntropyLoss() # can change this to another loss function
test_losses = []

In [45]:
def train_loop(dataloader, model, loss_fn, optimizer):
    avg_loss = 0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        optimizer.zero_grad()
                  
        # print(X)
        pred = model(X)
        # print(f"pred:{pred}")
        # print(f"target:{y}")
    
        # print(f"pred: {pred}")
        # print(f"y shape:{y.shape}")
        loss = loss_fn(pred, y)
        avg_loss += loss
        loss.backward()
        optimizer.step()
        
        # if batch % 100 == 0:
        #     print(f"loss: {loss:>7f}")
    avg_loss = avg_loss / size
    print(f"average loss: {avg_loss}")

        # if batch % 50 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [46]:
size = len(train_dataloader.dataset)
print(size)

7500


In [47]:
def test():
    avg_accuracy = 0
    size = len(test_dataloader.dataset)
    i = 0
    with torch.no_grad():
        for data, target in test_dataloader:
            output = model(data)
            # print(output*255)
            for out_i in range(100):
                
                masked_arr = mask(np.array(data.cpu().data[out_i][0]), np.array(output.cpu().data[out_i][0]))
                num_dots = np.count_nonzero(masked_arr != 0.0)
                num_expected = test_num_points[i]
                
                curr_diff = (num_dots - num_expected)**2
                if (out_i % 50 == 0):
                    print(f"diff: {curr_diff}")
                # num_expected = np.count_nonzero(np.array(target.cpu().data[out_i][0]) == 0.0)

                # print(np.array(target.cpu().data[out_i][0]).shape)
                # if (i == t):
                #     print(f"num counted: {num_dots}, num expected:{num_expected}")

                #     im = Image.fromarray(np.array(data.cpu().data[out_i][0])*255)
                #     im.show()

                # print(f"accuracy of this trial: {(num_dots - num_expected)/num_expected * 100}")
                avg_accuracy += curr_diff
                i += 1
    avg_accuracy = math.sqrt(avg_accuracy / size)
    print(f"RMSE accuracy: {avg_accuracy}")

# test(500)

In [48]:
# print(test_num_points[500])

In [49]:
# test()
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    # test(500)
    # test_loop(test_dataloader, model, loss_fn)
    
print("Done!")


Epoch 1
-------------------------------
average loss: 0.0010806085774675012
Epoch 2
-------------------------------
average loss: 6.735212082276121e-05
Epoch 3
-------------------------------
average loss: 2.4535034754080698e-05
Epoch 4
-------------------------------
average loss: 1.9509603589540347e-05
Epoch 5
-------------------------------
average loss: 1.7081127225537784e-05
Epoch 6
-------------------------------
average loss: 1.5627543689333834e-05
Epoch 7
-------------------------------
average loss: 1.4375136743183248e-05
Epoch 8
-------------------------------
average loss: 1.3296492397785187e-05
Epoch 9
-------------------------------
average loss: 1.2266623343748506e-05
Epoch 10
-------------------------------
average loss: 1.1409091712266672e-05
Epoch 11
-------------------------------
average loss: 1.0580390153336339e-05
Epoch 12
-------------------------------
average loss: 9.792831406230107e-06
Epoch 13
-------------------------------
average loss: 9.050773769558873e-06

In [50]:
torch.save(model, 'model.pt')

In [51]:
# Model class must be defined somewhere
model = torch.load('model.pt')

In [63]:
def threshold_input(model_input):
    WHITE = 1
    BLACK = 0
    THRESHOLD = 0.5

    model_input[model_input != BLACK] = WHITE

In [64]:
def threshold_output(model_output):
    WHITE = 1
    BLACK = 0
    THRESHOLD = 0.5
    model_output[model_output <= THRESHOLD] = BLACK

In [65]:
def mask(thresholded_input, model_output):
    WHITE = 1
    # BLACK = 0
    # THRESHOLD = 0.5

    # model_output = 1-model_output
    # model_input = 1-model_input

    # model_output[model_output > 0.5] = WHITE
    # model_input[model_output > 0.5] = WHITE

    # # print (model_input)

    # # for i in model_input:
    # #     for j in i:
    # #         if j  1:
    # #             print (j)

    # input = Image.fromarray(model_input * 255)
    # input.show()
    

    masked_arr = np.logical_or(thresholded_input, model_output)
    
    # print(m)
    return masked_arr

In [69]:
i = randint(0,100)
print (i)

with torch.no_grad():
    for data, target in test_dataloader:
        raw_output = model(data)    
        normOutput = (raw_output-raw_output.min())/(raw_output.max()-raw_output.min())

        # INPUT
        input_arr = np.array(data.cpu().data[i][0])
        input = Image.fromarray(input_arr * 255)
        # input.show()

        # OUTPUT
        output_arr = np.array(normOutput.cpu().data[i][0])
        output = Image.fromarray(output_arr * 255)
        output.show()
    
        threshold_input(input_arr)
        threshold_output(output_arr)
        input = Image.fromarray(input_arr * 255)
        input.show()

        masked_arr = mask(input_arr, output_arr)
        # print (masked_arr.astype(float))
        masked = Image.fromarray((masked_arr * 255).astype(np.uint8))
        masked.show()

        num_dots = np.count_nonzero(masked_arr == 0.0)
        num_expected = test_num_points[i]
        print(f"num counted: {num_dots}, num expected:{num_expected}")

        break

61
num counted: 9991, num expected:7.0



(eog:1303216): GLib-GObject-CRITICAL **: 17:38:27.683: g_object_unref: assertion 'G_IS_OBJECT (object)' failed

(eog:1303216): EOG-WARNING **: 17:38:27.683: Error when getting information for file “/tmp/.gnome_desktop_thumbnail.UKICW1”: No such file or directory

(eog:1303216): EOG-CRITICAL **: 17:38:27.710: eog_image_get_file: assertion 'EOG_IS_IMAGE (img)' failed

(eog:1303216): GLib-GIO-CRITICAL **: 17:38:27.710: g_file_equal: assertion 'G_IS_FILE (file1)' failed


In [55]:
# test()
# test()
# for t in range(epochs):
#     print(f"Epoch {t+1}\n-------------------------------")
#     # train_loop(train_dataloader, model, loss_fn, optimizer)
#     test()
#     # test_loop(test_dataloader, model, loss_fn)
    
# print("Done!")
